In [1]:
import pandas as pd

In [2]:
all_muts_df = pd.read_pickle("./data/2_4_df.pkl")
display(all_muts_df.shape, all_muts_df.head())

(2641, 31)

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,...,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links
0,2,R110G (CGT→GGT),clsA,SNP,1308318,NaN,G→C,1,42C,124,...,{},{},{},True,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']},[],{}
1,6,,rph,DEL,3815859,NaN,Δ82 bp,1,42C,124,...,{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK120000854'], 'ECK1200007...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000079...",[],{}
2,7,A734V (GCG→GTG),rpoC,SNP,4187550,NaN,C→T,1,42C,124,...,{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{}
3,8,D9A (GAT→GCT),hfq,SNP,4400313,NaN,A→C,1,42C,124,...,{},{},{},True,"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",{'ECK120000431': ['ECK120000431']},[{'name': 'yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC...,{'ECK125147410': ['ECK120000431']},[{'name': 'CRP'}],{'CRP': ['yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC']}
4,0,coding (380‑400/1149 nt),nagA,DEL,702352,NaN,Δ21 bp,1,42C,124,...,{},{},{},True,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']},"[{'name': 'CRP'}, {'name': 'NagC'}, {'name': '...","{'CRP': ['nagBAC-umpH'], 'NagC': ['nagBAC-umpH..."


In [3]:
TU_objects_df = pd.read_pickle("./data/TU_objects_df.pkl")
gene_synonym_df = pd.read_pickle("./data/gene_synonym_df.pkl")
gene_pathway_df = pd.read_pickle("./data/gene_pathway_df.pkl")

In [4]:
from util.gene import get_gene_bnum


def _get_pathways(gene_id):
    annots = []
    bnum = get_gene_bnum(gene_id, gene_synonym_df)
    df = gene_pathway_df[gene_pathway_df["refseq_locus_tag"]==bnum]
    for pathway in df["pathway_name"]:
        d = {"name": pathway}
        annots.append(d)
    return annots


def get_pathways(mut_row):
    annots = []
    for feat_d in mut_row["genomic features"]:
        # !!! This part only handles mutations that land within a genomic feature.
        # Need to also implement logic for handling mutations that land within intergenic regions.
        if feat_d["genetic"]:
            annots += _get_pathways(feat_d["RegulonDB ID"])
        # This will catch whether no features exist in the feat_ID_set
        # or if those features are just integenic regions.
        # Get operon genetic feature link dict through overlapping mutation and operon ranges.
        # All genetic features are annotated within both the "genetic features" and "genetic feature links" column.
        # If no integenic genomic feature is hit by a mutation, use the genetic feature range in an overlap analysis with operons.
        else:
            TU_IDs = set(TU_objects_df[TU_objects_df["TU_OBJECT_ID"]==feat_d["RegulonDB ID"]]["TRANSCRIPTION_UNIT_ID"])
            for TU_ID in TU_IDs:
                subset_TU_genes_df = TU_objects_df[(TU_objects_df["TRANSCRIPTION_UNIT_ID"]==TU_ID) & (TU_objects_df["TU_OBJECT_CLASS"]=='GN')]
                for gene_id in subset_TU_genes_df["TU_OBJECT_ID"].drop_duplicates():
                    if str(gene_id) != 'nan' and gene_id != '':  # RegulonDB can return 'nan' or '' gene names
                        annots += _get_pathways(gene_id)
    return annots


all_muts_df["pathways"] = all_muts_df.apply(get_pathways, axis=1)
all_muts_df.head()

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,...,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,pathways
0,2,R110G (CGT→GGT),clsA,SNP,1308318,NaN,G→C,1,42C,124,...,{},{},True,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']},[],{},[]
1,6,,rph,DEL,3815859,NaN,Δ82 bp,1,42C,124,...,{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK120000854'], 'ECK1200007...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000079...",[],{},"[{'name': 'Pyrimidine metabolism'}, {'name': '..."
2,7,A734V (GCG→GTG),rpoC,SNP,4187550,NaN,C→T,1,42C,124,...,{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{},"[{'name': 'Purine metabolism'}, {'name': 'Pyri..."
3,8,D9A (GAT→GCT),hfq,SNP,4400313,NaN,A→C,1,42C,124,...,{},{},True,"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",{'ECK120000431': ['ECK120000431']},[{'name': 'yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC...,{'ECK125147410': ['ECK120000431']},[{'name': 'CRP'}],{'CRP': ['yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC']},[]
4,0,coding (380‑400/1149 nt),nagA,DEL,702352,NaN,Δ21 bp,1,42C,124,...,{},{},True,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']},"[{'name': 'CRP'}, {'name': 'NagC'}, {'name': '...","{'CRP': ['nagBAC-umpH'], 'NagC': ['nagBAC-umpH...","[{'name': 'Galactose metabolism'}, {'name': 'A..."


In [5]:
TU_df = pd.read_pickle("./data/TU_df.pkl")
operon_df = pd.read_pickle("./data/operon_df.pkl")
operon_df["range"] = operon_df.apply(lambda row: (row["REGULATIONPOSLEFT"], row["REGULATIONPOSRIGHT"]), axis=1)

In [6]:
from util.operon import get_operon_ID_set


def _add_links_to_d(bnum, d, op_ID_set):
    for pathway in gene_pathway_df[gene_pathway_df["refseq_locus_tag"]==bnum]["pathway_name"]:
        if pathway not in d.keys(): d[pathway] = []
        d[pathway] += list(op_ID_set)


def get_pathway_operon_links_d(mut_row):
    pathway_operon_d = dict()
    for feat_d in mut_row["genomic features"]:
        if feat_d["genetic"]:
            op_ID_set = get_operon_ID_set(feat_d["RegulonDB ID"], TU_objects_df, TU_df, operon_df)
            if str(feat_d["RegulonDB ID"]) != 'nan' and feat_d["RegulonDB ID"] != '':
                bnum = get_gene_bnum(feat_d["RegulonDB ID"], gene_synonym_df)
                _add_links_to_d(bnum, pathway_operon_d, op_ID_set)
        else:
            TU_IDs = set(TU_objects_df[TU_objects_df["TU_OBJECT_ID"]==feat_d["RegulonDB ID"]]["TRANSCRIPTION_UNIT_ID"])
            for TU_ID in TU_IDs:
                subset_TU_genes_df = TU_objects_df[(TU_objects_df["TRANSCRIPTION_UNIT_ID"]==TU_ID) & (TU_objects_df["TU_OBJECT_CLASS"]=='GN')]

                op_ID_set = set()
                for _, TU_gene in subset_TU_genes_df.iterrows():
                     op_ID_set = op_ID_set | get_operon_ID_set(TU_gene["TU_OBJECT_ID"], TU_objects_df, TU_df, operon_df)

                for gene_id in subset_TU_genes_df["TU_OBJECT_ID"].drop_duplicates():
                    if str(gene_id) != 'nan' and gene_id != '':  # RegulonDB can return 'nan' or '' gene names
                        bnum = get_gene_bnum(gene_id, gene_synonym_df)
                        _add_links_to_d(bnum, pathway_operon_d, op_ID_set)
    return pathway_operon_d

                    
all_muts_df["pathway links"] = all_muts_df.apply(get_pathway_operon_links_d, axis=1)
all_muts_df.head()

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,...,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,pathways,pathway links
0,2,R110G (CGT→GGT),clsA,SNP,1308318,NaN,G→C,1,42C,124,...,{},True,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']},[],{},[],{}
1,6,,rph,DEL,3815859,NaN,Δ82 bp,1,42C,124,...,{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK120000854'], 'ECK1200007...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000079...",[],{},"[{'name': 'Pyrimidine metabolism'}, {'name': '...","{'Pyrimidine metabolism': ['ECK120014627'], 'D..."
2,7,A734V (GCG→GTG),rpoC,SNP,4187550,NaN,C→T,1,42C,124,...,{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{},"[{'name': 'Purine metabolism'}, {'name': 'Pyri...","{'Purine metabolism': ['ECK120016992'], 'Pyrim..."
3,8,D9A (GAT→GCT),hfq,SNP,4400313,NaN,A→C,1,42C,124,...,{},True,"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",{'ECK120000431': ['ECK120000431']},[{'name': 'yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC...,{'ECK125147410': ['ECK120000431']},[{'name': 'CRP'}],{'CRP': ['yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC']},[],{}
4,0,coding (380‑400/1149 nt),nagA,DEL,702352,NaN,Δ21 bp,1,42C,124,...,{},True,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']},"[{'name': 'CRP'}, {'name': 'NagC'}, {'name': '...","{'CRP': ['nagBAC-umpH'], 'NagC': ['nagBAC-umpH...","[{'name': 'Galactose metabolism'}, {'name': 'A...","{'Galactose metabolism': ['ECK125162049'], 'Am..."


In [7]:
all_muts_df.to_pickle("./data/2_5_df.pkl")